In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import pystan, pickle, popmachine

from pystan_cache.pystan_cache import caching_stan

In [ ]:
machine = popmachine.Machine('sqlite:///../popmachine_local/.popmachine.db')

In [ ]:
filter(lambda x: 'PQ' in x, machine.plates(names=True))

In [ ]:
plates = [
 u'20150517 PQ 3',
 u'20150715 PQ 8',
 u'20150702 PQ 6',
 u'20150514 PQ 2',
#  u'20150409 PQ test',
 u'20150630 PQ 5',
 u'20150607 PQ 4',
 u'20150704 PQ 7',
 u'20150717 PQ 9',
 u'20161010_PQ_osmo',
 u'20161107_PQ_osmo_combo',
 u'20161024 PQ crosscomp',
 u'20150828 PQ glutamate']

In [ ]:
# ds = machine.search(include=['Strain'], **{'mM PQ':[0.0, .333, 333.0]})
ds = machine.search(plates = plates, include=['Strain'], **{'mM PQ':[0.0, .333, 333.0], 'M NaCl':[4.2, None]})
ds.meta.Strain.unique()

In [ ]:
ds = machine.search(plates = plates, Strain=['ura3', 'rosR'], **{'mM PQ':[0.0, .333], 'M NaCl':[4.2, None]})
ds.log()
ds.filter()
ds.trim(5)
ds.poly_scale(2, groupby=['plate', 'mM PQ'])

ds.data = ds.data.iloc[::3,:]

In [ ]:
plt.figure(figsize=(12,4))
ds.plot(columns=['plate'], colorby=['mM PQ'], rows=['Strain'])

In [ ]:
ds.meta.Strain.unique()

In [ ]:
ds.meta.plate.unique()

In [ ]:
x,y, design, labels = ds.build(['mM PQ', 'Strain', 'plate'],scale=True)

In [ ]:
y = (y-y.mean())/y.std()
x = (x-x.mean())/x.std()

In [ ]:
design

In [ ]:
y.shape, x.shape

In [ ]:
labels

In [ ]:
dm = np.zeros((y.shape[1], 4 + 4*ds.meta.plate.unique().shape[0]))
dm[:,0] = 1
# dm[:,1] = 1 - 2*design['mM PQ']
# dm[:,1] = 1 - 2*(design['mM PQ'] == labels[0].index('0.0'))
dm[:,1:3] = design[['mM PQ', 'Strain']].values
dm[:,3] = dm[:,1] * dm[:,2]

for i in range(design.plate.unique().shape[0]):
    dm[:,4+i*4:8+i*4] = dm[:,:4] * (design.plate==i).values[:,None]

In [ ]:
plt.imshow(dm, aspect='auto')

In [ ]:
dm = dm[:,(dm!=0).any(0)]
plt.imshow(dm, aspect='auto')

In [ ]:
gp_multi = caching_stan.stan_model(file='models/gp_multi.stan')

In [ ]:
# random effect

p = dm.shape[1]
n = x.shape[0]

# design = np.zeros((p, 1+p))
# design[:,0] = 1
# design[:,1:] = np.eye(p)

priors = [1, 2, 3, 4] + [5,6,7,8] * ds.meta.plate.unique().shape[0]

sim_data = {
    'N': n,
    'P':y.shape[1],
    'K':dm.shape[1],
    'L': max(priors),
    'prior':priors,     
    'design': dm #[[1,1,0,0],[1,0,1,0],[1,0,0,1]]
}

In [ ]:
import scipy

In [ ]:
z = np.linspace(0, 10)

plt.plot(z, scipy.stats.gamma.pdf(z, 1.5, scale=.4))
plt.plot(z, scipy.stats.gamma.pdf(z, 1.5, scale=2))

# plt.semilogx()

In [ ]:
train_data = sim_data.copy()
train_data['N'] = x.shape[0]
train_data['y'] = y.T
train_data['x'] = x[:,0]

train_data['alpha_prior'] = [[1,1]] * 4 + [[.1,1]] * dm.shape[1] - 4
train_data['length_scale_prior'] = [[1.5,2]] * dm.shape[1]

train_data

In [ ]:
tsamples = gp_multi.sampling(data=train_data, chains=2, iter=2000, control = {'adapt_delta': 0.8})

In [ ]:
tsamples

In [ ]:
plt.figure(figsize=(10,4))
tsamples.traceplot(['length_scale', 'alpha', 'sigma','lp__'])
plt.tight_layout()

In [ ]:
tsamp = tsamples.extract(permuted=True)

In [ ]:
ncol = 2
nrow = int(1.*(dm.shape[1]-1)/ncol) + 1

plt.figure(figsize=(4*ncol, 4*nrow))

for i in range(dm.shape[1]):
    
    plt.subplot(dm.shape[1]/ncol + 1, ncol, i + 1)
    
    plt.plot(x, tsamp['f'][:,i,:].mean(0),)
    #plt.plot(x, tsamp['f'][:,i,:].T,c='k', alpha=.2)
    plt.fill_between(x[:,0], 
                     tsamp['f'][:,i,:].mean(0)-2*tsamp['f'][:,i,:].std(0),
                     tsamp['f'][:,i,:].mean(0)+2*tsamp['f'][:,i,:].std(0),alpha=.1)
    plt.plot([x.min(), x.max()], [0, 0], lw=3, c='k')
    
    if i > 1:
        plt.ylim(-.41, .41)
    
    #plt.plot(x[train_ind],f[i,train_ind].T,'--')

In [ ]:
temp = tsamp['f'][:,3::2].sum(1)

plt.plot(x, temp.mean(0),)
#plt.plot(x, tsamp['f'][:,i,:].T,c='k', alpha=.2)
plt.fill_between(x[:,0], 
                 temp.mean(0)-2*temp.std(0),
                 temp.mean(0)+2*temp.std(0),alpha=.1)

In [ ]:
plt.plot(tsamp['alpha'],alpha=.4)
plt.semilogy()
plt.legend()

In [ ]:
tsamp['alpha'].shape

In [ ]:
for i in range(4):

    temp = tsamp['alpha'][:,i]

    z = np.linspace(temp.min()*.7, temp.max()*1.3)
    kde = scipy.stats.gaussian_kde(temp)


    plt.plot(z, kde.pdf(z), label='alpha %d'%i)

plt.semilogx()    
plt.legend()

In [ ]:
for i in range(4):

    temp = tsamp['alpha'][:,i]

    z = np.linspace(temp.min(), temp.max())
    kde = scipy.stats.gaussian_kde(temp)

    plt.plot(z, kde(z)/kde(z).max(), label='alpha %d'%i)
    
temp = tsamp['sigma']
z = np.linspace(temp.min(), temp.max())
kde = scipy.stats.gaussian_kde(temp)

plt.plot(z, kde(z)/kde(z).max(), label='sigma')

plt.semilogx()    
plt.legend()

In [ ]:
for i in range(4):

    temp = tsamp['length_scale'][:,i]

    z = np.linspace(temp.min()*.7, temp.max()*1.3)
    kde = scipy.stats.gaussian_kde(temp)


    plt.plot(z, kde(z), label='length_scale %d'%i)

plt.semilogx()    
plt.legend()

# null model 

In [ ]:
null_train_data = train_data.copy()
null_train_data['design'] = train_data['design'][:,:2]
null_train_data['prior'] = null_train_data['prior'][:2]
null_train_data['length_scale_prior'] = null_train_data['length_scale_prior'][:2]
null_train_data['alpha_prior'] = null_train_data['alpha_prior'][:2]
null_train_data['K'] = null_train_data['L'] = 2

In [ ]:
nullSamples = gp_multi.sampling(data=null_train_data, chains=2, iter=2000, control = {'adapt_delta': 0.8})

In [ ]:
nullSamples

In [ ]:
plt.figure(figsize=(10,4))
nullSamples.traceplot(['length_scale', 'alpha', 'sigma','lp__'])
plt.tight_layout()

In [ ]:
nsamp = nullSamples.extract(permuted=True)

In [ ]:
ncol = 2
nrow = int(1.*(dm.shape[1]-1)/ncol) + 1

plt.figure(figsize=(4*ncol, 4*nrow))

for i in range(2):
    
    plt.subplot(dm.shape[1]/ncol + 1, ncol, i + 1)
    
    plt.plot(x, nsamp['f'][:,i,:].mean(0),)
    #plt.plot(x, tsamp['f'][:,i,:].T,c='k', alpha=.2)
    plt.fill_between(x[:,0], 
                     nsamp['f'][:,i,:].mean(0)-2*nsamp['f'][:,i,:].std(0),
                     nsamp['f'][:,i,:].mean(0)+2*nsamp['f'][:,i,:].std(0),alpha=.1)
    plt.plot([x.min(), x.max()], [0, 0], lw=3, c='k')
    
    if i > 1:
        plt.ylim(-.48, .48)
    
    #plt.plot(x[train_ind],f[i,train_ind].T,'--')

In [ ]:
temp = tsamp['f'][:,1]

plt.plot(x, temp.mean(0),)
plt.fill_between(x[:,0], 
                 temp.mean(0)-2*temp.std(0),
                 temp.mean(0)+2*temp.std(0),alpha=.1)

temp = nsamp['f'][:,1]

plt.plot(x, temp.mean(0),)
plt.fill_between(x[:,0], 
                 temp.mean(0)-2*temp.std(0),
                 temp.mean(0)+2*temp.std(0),alpha=.1)

plt.plot([x.min(), x.max()], [0, 0], lw=3, c='k')